### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [14]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [29]:
importlib.reload(train)

models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [32]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    # "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    # "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.54141582 0.53765574 0.53858227 0.54016306 0.54212183 0.50607578
 0.51842087 0.53486696 0.53574133 0.53791047 0.5124806  0.51532415
 0.51395124 0.5112998  0.52372181 0.52854533 0.54076518 0.55297455
 0.55466285 0.55424836 0.55385331 0.55653555 0.56203275 0.56485263
 0.56760288        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training LGB ...
Model LGB, Actual: [0 1], [47 72], Predictions: [0 1], [59 60]
Training RF ...
Model RF, Actual: [0 1], [47 72], Predictions: [0 1], [74 45]
Training XGB ...
Model XGB, Actual: [0 1], [47 72], Predictions: [0 1], [73 46]

[('bpm', 169), ('lf_rr', 167), ('rmssd', 161), ('hf_rr', 158), ('sdnn', 144)]


[('rmssd', 0.2197703198019751), ('hf_rr', 0.21603479764085978), ('bpm', 0.19188415828155617), ('sdnn', 0.19168782366885564), ('lf_rr', 0.18062290060675337)]


[('rmssd', 0.23064017), ('hf_rr', 0.21437936), ('sdnn', 0.21267723), ('bpm', 0.17328818), ('lf_rr', 0.16901508)]

Grid search for LGB ...
Gri

One or more of the test scores are non-finite: [0.52483379 0.54429606 0.53540799 0.5479231  0.54100854 0.54716302
 0.55682406 0.56001065 0.56061025 0.55953161 0.52823379 0.54586416
 0.54824542 0.55876366 0.56276207 0.53112762 0.53195516 0.53819776
 0.53852851 0.54679072 0.56218463 0.55031562 0.5619018  0.5618811
 0.5672631         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training LGB ...
Model LGB, Actual: [0 1], [91 48], Predictions: [0 1], [74 65]
Training RF ...
Model RF, Actual: [0 1], [91 48], Predictions: [0 1], [88 51]
Training XGB ...
Model XGB, Actual: [0 1], [91 48], Predictions: [0 1], [76 63]

[('rmssd', 122), ('bpm', 120), ('hf_rr', 107), ('lf_rr', 96), ('sdnn', 94)]


[('rmssd', 0.21641039872400364), ('hf_rr', 0.20310372804406424), ('sdnn', 0.2004859975764841), ('lf_rr', 0.195504720800605), ('bpm', 0.18449515485484316)]


[('lf_rr', 0.2500518), ('rmssd', 0.22112334), ('sdnn', 0.19401747), ('hf_rr', 0.17746802), ('bpm', 0.15733941)]

Grid search for LGB ...
Grid sea

...

Grid search for XGB ...
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training LGB ...
Model LGB, Actual: [0 1], [104  26], Predictions: [0 1], [68 62]
Training RF ...
Model RF, Actual: [0 1], [104  26], Predictions: [0 1], [57 73]
Training XGB ...
Model XGB, Actual: [0 1], [104  26], Predictions: [0 1], [65 65]

[('bpm', 123), ('rmssd', 123), ('hf_rr', 117), ('lf_rr', 99), ('sdnn', 68)]


[('rmssd', 0.2259874572382313), ('sdnn', 0.20147056166508293), ('bpm', 0.19808069008299845), ('hf_rr', 0.19198028945560922), ('lf_rr', 0.18248100155807806)]


[('hf_rr', 0.24679269), ('rmssd', 0.21559526), ('lf_rr', 0.2061859), ('sdnn', 0.16641009), ('bpm', 0.16501613)]

Grid search for LGB ...


One or more of the test scores are non-finite: [0.46678819 0.48973787 0.49115963 0.47826861 0.49231085 0.45310377
 0.4575542  0.46287744 0.46098378 0.46824104 0.47631178 0.48455309
 0.48336985 0.48944498 0.49141128 0.48381815 0.49088496 0.48900915
 0.49684292 0.50413791 0.48033158 0.48344606 0.48028169 0.48590263
 0.48784568        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training LGB ...
Model LGB, Actual: [0 1], [35 83], Predictions: [0 1], [80 38]
Training RF ...
Model RF, Actual: [0 1], [35 83], Predictions: [0 1], [73 45]
Training XGB ...
Model XGB, Actual: [0 1], [35 83], Predictions: [0 1], [73 45]

[('rmssd', 241), ('bpm', 229), ('lf_rr', 227), ('hf_rr', 226), ('sdnn', 185)]


[('rmssd', 0.21991121390320642), ('hf_rr', 0.20691124112427037), ('bpm', 0.20239996255792955), ('lf_rr', 0.18935133612115745), ('sdnn', 0.1814262462934363)]


[('lf_rr', 0.37845594), ('hf_rr', 0.16664502), ('bpm', 0.16316761), ('rmssd', 0.16075867), ('sdnn', 0.13097274)]

Grid search for LGB ...
Gr

One or more of the test scores are non-finite: [0.55711078 0.55792938 0.55702169 0.56468118 0.56584974 0.57548418
 0.58134415 0.58367521 0.57154792 0.56910925 0.56086271 0.57404759
 0.57746676 0.57016158 0.58455343 0.59910495 0.60966514 0.5956941
 0.59218448 0.58984993 0.56113752 0.57945482 0.57757677 0.57499502
 0.57245954        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training LGB ...
Model LGB, Actual: [0 1], [117  33], Predictions: [0 1], [114  36]
Training RF ...
Model RF, Actual: [0 1], [117  33], Predictions: [0 1], [104  46]
Training XGB ...
Model XGB, Actual: [0 1], [117  33], Predictions: [0 1], [84 66]

[('bpm', 196), ('rmssd', 171), ('sdnn', 156), ('hf_rr', 155), ('lf_rr', 148)]


[('rmssd', 0.23032449886796097), ('hf_rr', 0.2063824000213445), ('lf_rr', 0.1933281023342739), ('sdnn', 0.185279918575493), ('bpm', 0.1846850802009276)]


[('rmssd', 0.27880675), ('lf_rr', 0.24949512), ('hf_rr', 0.1861591), ('sdnn', 0.15080844), ('bpm', 0.13473058)]

Model evaluation metri